# Interesect peaks and SNPs
Obtain list of SNPs that overlap genomic features of iPSCs and iPSC-CMs

In [70]:
home="/frazer01/home/paola/Family1070/private_output/PeakCalling"
setwd(home)

In [71]:
marks<-c('ATAC_CM', 'ATAC_IPSC', 'H3K27AC_CM', 'H3K27AC_IPSC' , 'NKX25')

In [75]:
######################################################################################
# Get and prepare genotype tables including snp +/- 200 bp outsied the peak
######################################################################################
system("module load cardips")
system("cd /frazer01/home/paola/Family1070/private_output/PeakCalling")

for ( m in 1:length(marks)){
    mark    = marks[m]
    setwd(mark)
    command  = "awk -v OFS=\"\t\" '$1~/^chr[0-9]+$/ || $1==\"chrX\" || $1==\"chrY\" {print $0}' meta_macs2_callPeak_peaks.q001.narrowPeak.collapse.sorted.bed > tmp"
    
    command0 = "mv tmp meta_macs2_callPeak_peaks.q001.narrowPeak.collapse.sorted.bed"
    
    command1 = paste ("awk -v OFS=\"\t\" '{$4=$1\":\"$2\"-\"$3;$5=$2-200; $6=$3+200; print substr($1,4),$5,$6,$4}' meta_macs2_callPeak_peaks.q001.narrowPeak.collapse.sorted.bed > ", 
                       mark, "_sorted.p200.bed", sep="" )
    command2 = paste ("bedtools intersect -a /frazer01/projects/CARDIPS/analysis/family1070/sevenIndi.phased.variant.gt -b ", mark, "_sorted.p200.bed",
                      " -wo > " , mark, "_p200_variants.txt", sep="" ) 
    command3 = paste ("grep rs ", mark, "_p200_variants.txt > ", mark, "_p200_rs.variants.txt", sep="" )     
    
    system(command)
    system(command0)
    system(command1)
    system(command2)
    system(command3)
    setwd(home)
}



# zcat /projects/CARDIPS/analysis/family1070/sevenIndi.phased.variant.vcf.gz | grep "#CHROM" > header.txt


In [73]:
files=paste(marks, "_p200_variants.txt", sep="" )
outfiles=paste(marks, "_p200_filtered_variants.txt", sep="" )

In [76]:
for (f in 1:length(marks)) {
     
     mark    = marks[f]
     snp     = read.table(paste(mark, files[f], sep="/"), stringsAsFactors=FALSE)
     cat(files[f], "\n")
colnames(snp)=c("CHROM","POS","POS+1","ID","REF","ALT","QUAL","iPSCORE_2_2","iPSCORE_2_4","iPSCORE_2_3","iPSCORE_2_6",
                "iPSCORE_2_7","iPSCORE_2_9","iPSCORE_2_1","peakCHR","peakStart-200", "peakEnd+200", "peakID", "ov")
snp=snp[,c(1:7, 14,8,10,9,11,12,13,15:18)]
gt=snp

gt <- replace(gt, gt == "1/1", 2)
gt <- replace(gt, gt == "1|1", 2)
gt <- replace(gt, gt == "0/1", 1)
gt <- replace(gt, gt == "0|1", 1)
gt <- replace(gt, gt == "1/0", 1)
gt <- replace(gt, gt == "1|0", 1)
gt <- replace(gt, gt == "0|0", 0)
gt <- replace(gt, gt == "0/0", 0)

#keep only bi-allelic SNps
sel=(gt[,8:14] == 1 | gt[,8:14] ==2 | gt[,8:14] == 0)
gt=subset(gt, rowSums(sel) ==7)

##calculate allelic differences
gt[,8:14]=sapply(gt[,8:14], as.numeric)


for (i in 8:13) { 
  for (j in (i+1):14) {
    
    name=paste(colnames(gt)[i],"vs",colnames(gt)[j], sep="")
    diff = gt[,i]-gt[,j]
    gt=cbind(gt, diff)
    colnames(gt)[length(gt)] = name
  }
}

#keep only SNps that are different in at least one couple 
no0=abs(gt[,19:39])>0
gt2=gt[rowSums(no0)>0, ]

var=NA
for (i in 1:nrow(gt2)) {
  var[i]<- paste("chr", gt2[i,1],":",gt2[i,2], sep="")}

gt2$varID=var

write.csv(gt2, paste(mark, outfiles[f], sep="/"), row.names=F)

}


ATAC_CM_p200_variants.txt 
ATAC_IPSC_p200_variants.txt 


# Intersect exons or promoters and SNPs


In [77]:
files = c( "/frazer01/home/paola/ASE/fam1070_exons_merged.variants.txt",
         "/home/paola/SNP_intersection/fam1070_promoters_by_gene.variants.txt")

output_names=c("exons_filtered_variants.csv", "promoters_filtered_variants.csv")
outdir='/home/paola/Family1070/private_output/Genic_snps/'

In [78]:
for (f in 2:2) {
    
snp=read.table(files[f], stringsAsFactors=FALSE)

if (f==1){
colnames(snp)=c("CHROM","POS","POS+1","ID","REF","ALT","QUAL","iPSCORE_2_2","iPSCORE_2_4","iPSCORE_2_3","iPSCORE_2_6",
                "iPSCORE_2_7","iPSCORE_2_9","iPSCORE_2_1","Feature.Chr","Feature.Start", "Feature.End", "ov")
        }
    
if (f==2){
colnames(snp)=c("CHROM","POS","POS+1","ID","REF","ALT","QUAL","iPSCORE_2_2","iPSCORE_2_4","iPSCORE_2_3","iPSCORE_2_6",
                "iPSCORE_2_7","iPSCORE_2_9","iPSCORE_2_1","Feature.Chr","Feature.Start", "Feature.End", "gene", "strand", "ov")
      }

snp=snp[,c(1:7, 14,8,10,9,11,12,13,15:length(snp))]
gt=snp

gt <- replace(gt, gt == "1/1", 2)
gt <- replace(gt, gt == "1|1", 2)
gt <- replace(gt, gt == "0/1", 1)
gt <- replace(gt, gt == "0|1", 1)
gt <- replace(gt, gt == "1/0", 1)
gt <- replace(gt, gt == "1|0", 1)
gt <- replace(gt, gt == "0|0", 0)
gt <- replace(gt, gt == "0/0", 0)

#keep only bi-allelic SNps
sel=(gt[,8:14] == 1 | gt[,8:14] ==2 | gt[,8:14] == 0)
gt=subset(gt, rowSums(sel) ==7)

##calculate allelic differences
gt[,8:14]=sapply(gt[,8:14], as.numeric)


for (i in 8:13) { 
  for (j in (i+1):14) {
    
    name=paste(colnames(gt)[i],"vs",colnames(gt)[j], sep="")
    diff = gt[,i]-gt[,j]
    gt=cbind(gt, diff)
    colnames(gt)[length(gt)] = name
  }
}

#keep only SNps that are different in at least one couple 
no0=abs(gt[,(length(snp)+1):(length(snp)+21)])>0
gt2=gt[rowSums(no0)>0, ]

var=NA
for (i in 1:nrow(gt2)) {
  var[i]<- paste("chr", gt2[i,1],":",gt2[i,2], sep="")}

gt2$varID=var

write.csv(gt2, paste(outdir,output_names[f], sep=""), row.names=F)

}